In [59]:
import tensorflow as TF
from tensorflow import keras
import matplotlib.pyplot as plt
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile(r"dogs-vs-cats.zip")
zip_ref.extractall('/content')
zip_ref.close()

In [60]:
train_ds = keras.utils.image_dataset_from_directory(
    directory=r"C:/Users/PMLS/OneDrive/Desktop/CAT vs DOG/BaseData/Training",
    labels='inferred',
    label_mode='int',         
    batch_size=32,
    image_size=(256, 256)     
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory=r"C:/Users/PMLS/OneDrive/Desktop/CAT vs DOG/BaseData/Validation", 
    labels='inferred',
    label_mode='int',           
    batch_size=32,
    image_size=(256, 256)      
)

# flow_from_directory is a method from ImageDataGenerator tha helps for labeling data to train our model.
# TARGET_SIZE is given bcs models need same size of every data.
# BATCH_SIZE is given bcs we need to train our model in batches(groups) for that we have to load some particular amount of data instead of complete data bcs it might be unfit for my memory.
# so in my case 1 batch = 40 images
# CLASS_MODEL = binary bcs we have to classify between 2 things.

Found 310 files belonging to 2 classes.
Found 340 files belonging to 2 classes.


In [61]:
#Normalize the dataset (0 and 1)
def Process(image, label):
    image = TF.cast(image/255., TF.float32)
    return image, label
train_ds = train_ds.map(Process)
validation_ds = validation_ds.map(Process)

In [67]:
# Creating CNN
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), padding='valid', activation='relu', input_shape = (256, 256, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Conv2D(64, kernel_size=(3,3), padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Conv2D(128, kernel_size=(3,3), padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Flatten())

model.add(Dense (128, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense (64, activation = 'relu'))
model.add(Dropout(0.01))
model.add(Dense (1, activation = 'sigmoid'))

In [68]:
model.summary()


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_26 (Conv2D)              │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 254, 254, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 125, 125, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_27 (MaxPooling2D) │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 60, 60, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,848,193 (56.64 MB)

 Trainable params: 14,847,745 (56.64 MB)

 Non-trainable params: 448 (1.75 KB)

In [69]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [70]:
history = model.fit(train_ds, epochs= 10, validation_data=validation_ds)

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 40s 4s/step - accuracy: 0.5525 - loss: 4.6995 - val_accuracy: 0.5618 - val_loss: 1.7399
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 43s 4s/step - accuracy: 0.7121 - loss: 2.5083 - val_accuracy: 0.5000 - val_loss: 8.8964
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 45s 5s/step - accuracy: 0.7861 - loss: 1.5343 - val_accuracy: 0.5000 - val_loss: 9.1615
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 42s 4s/step - accuracy: 0.8581 - loss: 0.8488 - val_accuracy: 0.5000 - val_loss: 8.4604
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 46s 5s/step - accuracy: 0.8721 - loss: 0.6251 - val_accuracy: 0.5000 - val_loss: 7.3356
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 40s 4s/step - accuracy: 0.8928 - loss: 0.5842 - val_accuracy: 0.5000 - val_loss: 7.6388
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 36s 4s/step - accuracy: 0.9327 - loss: 0.2535 - val_accuracy: 0.5000 - val_loss: 8.9343
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 34s 3s/step - accuracy: 0.9465 - loss: 0.4028 - val_accuracy: 0.5000 - val_loss: